### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [ ]:
from sklearn import linear_model, svm, kernel_ridge
from sklearn.metrics import mean_squared_error
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()
#regVols = linear_model.ElasticNet(positive = True)

### Root Mean Squared function

In [ ]:
def getRMSD (y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#w1 = [(3),(5),(10),(1)]
#w2 = [(2),(4),(6),(8)]
#getRMSD(w1,w2)

A function to calculate the RMSD was made; this was simple to construct as numpy had a function to calculate the mean square error.
This was then square rooted.

The aim was to use it to check how different a set of average volumes was compared to the estimations from the previous iteration, when this difference was of suitable value the loop would be stopped.

The areas which are commented out were used to test the function.

### Percentage function

In [ ]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

In [ ]:
#for chunk in np.array_split(unitformula, 10000):
#    print chunk
#for numb, chunk in unitformula.groupby(np.arange(len(unitformula)) // 10000):
#    print numb, chunk

This cell which is commented out was an early simple attempt at splitting the data into chunks

### Loop algorithm

In [ ]:
alpha = 0.00
elements = unitformula.columns.values
unitformula["Keep"] = 1
weights = []
alphas =[]
for chunk in zip(unitformula.groupby(np.arange(len(unitformula)) // 10000),vols.groupby(np.arange(len(vols)) // 10000),temps.groupby(np.arange(len(temps)) // 10000)):
    print "Chunk: ", chunk[0][0]
    prevalpha = 1
    RMSD = 6
    chunkformula = chunk[0][1]
    chunkvol = chunk[1][1]
    chunktemp = chunk[2][1]
    keepidx = chunkformula.index.tolist()
    i = 0
    while abs(prevalpha-alpha) > 0.00001 and (RMSD > 5) :
        print "Iteration:", i
        regVols.fit(chunkformula[chunkformula.Keep == 1], chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))
        if i > 0:
            prevw = w[0]
            #print "old w: ", prevw
        w = regVols.coef_
        if i > 0:
            RMSD = getRMSD(prevw,w[0])
            #print "New w", w[0]
            print "RMSD: ", RMSD
        x2=pd.DataFrame()
        for line in zip(elements, w[0]):
            x2[line[0]] = chunk[0][1][line[0]].apply(lambda x: x*line[1])
        x2["sum"] = x2.sum(axis=1)
        vol2 = chunkvol.loc[keepidx].div(x2["sum"].loc[keepidx], axis="index") - 1
        regAlpha.fit(chunktemp.loc[keepidx], vol2)
        if i > 0:
            prevalpha = alpha
            print "Previous Alpha: ", prevalpha
        alpha = regAlpha.coef_[0][0]
        print "Alpha: ", alpha
        print "deltaAlpha: ", abs(prevalpha-alpha)
        #print "dAlpha: ", abs(prevalpha - alpha)
        chunkformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
        keepidx = chunkformula.Keep[chunkformula.Keep == 1].index.tolist()
        #print "Element average volume (cubic angstroms)", zip(elements,w[0])
        i +=1
        if (len(keepidx) <= 1) or (i > 200):
            print "Failure"
            break
    weights.append(w[0])
    alphas.append(alpha)

This is similar to the previous loop; the new addition is that the loop now has another condition to be fulfilled; this is that the RMSD between the current average volumes and previous ones is less than 5.
To allow this to work if statements were added to the loop so that after the first iteration the RMSD and difference in alphas could be calculated.

One one occasion the mean alpha value was -0.00341222810177 which was still a poor result, even with the RMSD condition

Running with smaller chunk size seems to yield better results, but it wasn't tried till completion.

### Misc. - printing items

In [ ]:
print alphas
print np.mean(alphas)

Calculating the mean value of alpha

In [ ]:
print zip(elements,np.mean(weights, axis=1))

Outputting the average element volumes, over multiple iterations.